In [ ]:
#!/usr/bin/env python
import numpy as np
from math import ceil, floor
from numpy.linalg import det, inv, matrix_rank
from copy import copy
from cvxopt.glpk import ilp
from cvxopt import matrix
from itertools import permutations

dimensions = 4
max_minor = 3

def minor(arr, i=0):  #  ith row removed
    return arr[np.array(range(i)+range(i+1, arr.shape[0]))[:, np.newaxis],
               np.array(range(arr.shape[1]))]


def get_minors(A):
    return [det(minor(A,i)) for i in xrange(A.shape[0])]

diag = np.array([1]*(dimensions)).astype('i')
diag[-1] = max_minor
hermit = np.diag(diag)

for pre_last_row in permutations(xrange(dimensions), dimensions-1):
    hermit[dimensions-1][:-1] = list(pre_last_row)
    
    for restriction in permutations(xrange(-max_minor, max_minor + 1), dimensions):
        print restriction
#         restriction = np.array(restriction)
        restricted = np.append(hermit, np.ndarray(buffer=np.array(restriction), shape=(1, dimensions)), 0)
        minors = get_minors(restricted)
        if all(minors):
            continue
        abs_minors = [abs(m) for m in minors]
        min_abs_minor = min(abs_minors)
        max_abs_minor = max(abs_minors)
        
        c = restricted[-1]
        x = np.zeros(shape=(dimensions,1), dtype=float)
        x[-1] = float(dimensions)/float(max_minor)
        sigma = c.dot(x)  # ( b[n+1])
        f = copy(x)  # np.reshape(np.append(x, sigma), newshape=(dimensions + 1, 1))
        f[-1] = float(dimensions + 1)/float(max_minor)
        
        is_in_M = c.dot(f) > sigma
        target_f = matrix(-np.ones((dimensions + 1,1),dtype=float), tc='d')
        s = None
        if is_in_M:
            sigma = floor(sigma)
            while not s:
                (status,s) = ilp(target_f, 
                     G=matrix(restricted, tc='d'),
                     h=matrix(np.append(f, sigma), tc='d'),
                     I=set(xrange(dimensions)),
                    )
                print G
                print h
                if status != 'optimal':
                    print status
                    break
                sigma += 1
            sigma -= 1
        else:
            sigma = ceil(sigma)
            while not s:
                (status,s) = ilp(target_f, 
                     G=matrix(-restricted, tc='d'),
                     h=matrix(-np.append(f, sigma), tc='d'),
                     I=set(xrange(dimensions)),
                    )
                if status != 'optimal':
                    print status
                    break
                sigma -= 1
            sigma += 1
        print 'sigma after cycle:', sigma

(-3, -2, -1, 0)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, -1, 1)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, -1, 2)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, -1, 3)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, 0, -1)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, 0, 1)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, 0, 2)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, 0, 3)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, 1, -1)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, 1, 0)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, 1, 2)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, 1, 3)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, 2, -1)
LP relaxation is dual infeasible
sigma after cycle: nan
(-3, -2, 2, 0)
LP relaxation is dual infeasible
sigma after cycle: nan

/usr/local/lib/python2.7/dist-packages/numpy/linalg/linalg.py:1776: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
